## Implementation using TensorFlow on fashion minst dataset.

In [1]:
# ! pip install -U "ray[data,train,tune,serve]"

In [2]:
import ray
from ray import tune, train
import tensorflow as tf

In [3]:
ray.init()

2024-02-23 11:29:54,382	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.11.5
Ray version:,2.9.2
Dashboard:,http://127.0.0.1:8265


In [4]:
# Define a function to train the model
def train_model(config):
    from fashionmnist_model import FMM
    X_train, y_train, X_test, y_test = FMM.load_data()
    X_train, X_test = FMM.reshape_data(X_train, X_test) 
    
    # Define a TensorFlow model
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(config["units_1"], activation="relu"),
            tf.keras.layers.Dropout(config["dropouts_1"]),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(config["units_2"], activation="relu"),
            tf.keras.layers.Dropout(config["dropouts_2"]),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(10, activation="softmax"),
        ]
    )

    model.compile(
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    # Train the model
    model.fit(X_train, y_train, epochs=30, batch_size=config["batch_size"], verbose=0, validation_split=0.2)

    # Evaluate the model
    _, accuracy = model.evaluate(X_test, y_test)
    train.report({"test_accuracy": accuracy})

In [5]:
search_space = {
    "units_1": tune.grid_search([64, 128, 256]), 
    "units_2": tune.grid_search([64, 128, 256]),
    "dropouts_1": tune.grid_search([0.1, 0.5]),
    "dropouts_2": tune.grid_search([0.1, 0.5]),
    "batch_size": tune.grid_search([32, 64, 128]),
}

In [6]:
# Run hyperparameter tuning
analysis = tune.run(
    train_model,
    config=search_space,
    metric="test_accuracy",
    mode="max",
)

2024-02-23 11:29:54,989	INFO tune.py:583 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(raylet) [2024-02-23 11:30:04,336 E 58843 531688] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-02-23_11-29-52_330678_58831 is over 95% full, available space: 10911334400; capacity: 245107195904. Object creation will fail if spilling is required.
(train_model pid=58860) 2024-02-23 11:30:05.627844: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
(raylet) [2024-02-23 11:30:14,352 E 58843 531688] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-02-23_11-29-52_330678_58831 is over 95% full, available space: 9838391296; capacity: 245107195904. Object creation will fail if spilling is required.
(train_model pid=58859) 2024-02-23 11:30:06.072127: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz [repeated 7x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-de

243/313 [======================>.......] - ETA: 0s - loss: 0.3884 - accuracy: 0.8723


Trial name,test_accuracy
train_model_c6df2_00000,0.8735
train_model_c6df2_00001,0.8764
train_model_c6df2_00002,0.8707
train_model_c6df2_00003,0.855
train_model_c6df2_00004,0.8574
train_model_c6df2_00005,0.8594
train_model_c6df2_00006,0.869
train_model_c6df2_00007,0.8717
train_model_c6df2_00008,0.8628
train_model_c6df2_00009,0.8416


313/313 [==============================] - 1s 2ms/step - loss: 0.3912 - accuracy: 0.8594


...

(raylet) [2024-02-23 12:22:47,438 E 58843 531688] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-02-23_11-29-52_330678_58831 is over 95% full, available space: 9912172544; capacity: 245107195904. Object creation will fail if spilling is required.


313/313 [==============================] - 0s 924us/step - loss: 0.3582 - accuracy: 0.8821


(raylet) [2024-02-23 12:22:57,443 E 58843 531688] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-02-23_11-29-52_330678_58831 is over 95% full, available space: 10987159552; capacity: 245107195904. Object creation will fail if spilling is required.
(raylet) [2024-02-23 12:23:07,542 E 58843 531688] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-02-23_11-29-52_330678_58831 is over 95% full, available space: 10987106304; capacity: 245107195904. Object creation will fail if spilling is required.


313/313 [==============================] - 0s 626us/step - loss: 0.3740 - accuracy: 0.8720


2024-02-23 12:23:16,879	INFO tune.py:1042 -- Total run time: 3201.89 seconds (3199.43 seconds for the tuning loop).


In [7]:
ray.shutdown()

In [8]:
# Print the best hyperparameters and results
config = analysis.best_config
print("Best hyperparameters:", analysis.best_config)
print("Best accuracy:", analysis.best_result["test_accuracy"])

Best hyperparameters: {'units_1': 256, 'units_2': 256, 'dropouts_1': 0.1, 'dropouts_2': 0.1, 'batch_size': 64}
Best accuracy: 0.8867999911308289
